# Logistic Regression Example

Logistic regression implementation with TensorFlow v2 library.

This example is using a low-level approach to better understand all mechanics behind the training process.

## MNIST Dataset Overview

This example is using MNIST handwritten digits. The dataset contains 60,000 examples for training and 10,000 examples for testing. The digits have been size-normalized and centered in a fixed-size image (28x28 pixels) with values from 0 to 255. 

In this example, each image will be converted to float32, normalized to [0, 1] and flattened to a 1-D array of 784 features (28*28).

![MNIST Dataset](../img/mnist_100_digits.png)

In [16]:
from __future__ import absolute_import, division, print_function
import sys
sys.path.append('./')
import fox_proxy
import tensorflow as tf
import numpy as np


In [17]:
#test net
# import requests
# requests.get('http://www.google.com.tw')


In [18]:
# MNIST dataset parameters
num_classes = 10 # 0 to 9 digits
num_features = 28 * 28

# Training parameters
learning_rate = 0.01
training_steps = 1000
batch_size = 256
display_step = 50



In [19]:
# Prepare MNIST DATA 
from tensorflow.keras.datasets  import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
# to float 32
x_train, x_test = np.array(x_train,np.float32), np.array(x_test, np.float32)
# Flaten to 1-D vector ,784 features
x_train , x_test = x_train.reshape([-1, num_features]), x_test.reshape([-1, num_features])
#Normalize images value from [0, 255] to [0,1]
x_train, x_test = x_train / 255, x_test / 255


In [20]:
# shuffle and batch data, use tf.data
# ref https://stackoverflow.com/questions/53514495/what-does-batch-repeat-and-shuffle-do-with-tensorflow-dataset
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.repeat().shuffle(5000).batch(batch_size).prefetch(1)


In [21]:
# Weight of shape [780,10] the 28*28 image features and total number of classes
W = tf.Variable(tf.ones([num_features, num_classes]), name='weight')
# Bias of shape [10] the total number of classes
b = tf.Variable(tf.zeros([num_classes]), name='bias')


In [22]:
# Logisstic regression Wx+b
def logistic_regression(x):
    # Apply softmax to normalize the logits to a probability distribution
    return tf.nn.softmax(tf.matmul(x,W) + b)
# 
#Cross-entropy loss function
def cross_entropy(y_pred, y_true):
    # encode label to a one hot vector
    y_true = tf.one_hot(y_true, depth=num_classes)
    # Clip prediction value to avoid  log(0) error
    y_pred = tf.clip_by_value(y_pred,1e-9, 1)
    #Compute cross-entropy
    return tf.reduce_mean

# Accuracy metric
def accuracy(y_pred, y_true):
    # Predicted class is the index of highest score in prediction vector (i.e argmax)
    corrcet_prediction = tf.equal(tf.argmax(y_pred,1),tf.cast(y_true,tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction,tf.float32))

# Stochastic gradient descent optimizer
optimizer = tf.optimizers.SGD(learning_rate)


In [23]:
'''
tf.clip_by_value(A, min, max)：输入一个张量A，把A中的每一个元素的值都压缩在min和max之间。小于min的让它等于min，大于max的元素的值等于max。
'''

'\ntf.clip_by_value(A, min, max)：输入一个张量A，把A中的每一个元素的值都压缩在min和max之间。小于min的让它等于min，大于max的元素的值等于max。\n'

In [24]:
# Optimization process
def run_optimization(x, y):
    # Wrap computation inside a GradientTape fro automatic differentiation
    with tf.GradientTape() as g:
        pred = logistic_regression(x)
        loss = cross_entropy(pred, y)
    
    #Compute gradients
    gradients = g.gradient(loss,[W, b])
    # Update W and b following gradients
    optimizer.apply_gradients(zip(gradients,[W,b])) #=================?
    

In [25]:
# Run training for th e given number of steps
for step, (batch_x, batch_y) in enumerate(train_data.take(training_steps),1):
    #run the optimization to update W and b
    run_optimization(batch_x, batch_y)
    
    if step % display_steps == 0:
        pred = logistic_regression(batch_x)
        loss = cross_entropy(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print("step: %i , loss: %f, accuracy: %f " %(step, loss, acc))
        

NameError: name 'cross_entropy' is not defined